In [ ]:
!pip install openai faiss-cpu langchain tiktoken pypdf

In [ ]:
# Document Loading:
# ∟論文PDFのデータをロードする。
#  Splitting:
# ∟ドキュメントを指定サイズのスプリットに分割する。

In [ ]:
## PDFデータのロード
### ライブラリの読み込み
import os
import openai
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFLoader
from google.colab import userdata

### API keysのセット
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

### pdfファイルのロード
loader = PyPDFLoader("https://arxiv.org/pdf/2005.14165.pdf")
pages = loader.load_and_split() #PDFからテキストを抽出、抽出したテキストをページごとに分割

In [ ]:
#  Storage:
# ∟分割されたドキュメントをVectorStoreに格納する。

In [ ]:
## Embedding
### ライブラリの読み込み
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

### 関連するテキストの抽出
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(
    pages,
    embeddings# 文章の分散表現を作成するEmbeddingモデル
    )
vectorstore.save_local("../database/faiss_index")

In [ ]:
#  Retrieval:
# ∟ストレージからスプリットを取得する（例えば、多くの場合、入力された質問と同様の埋め込みを持つ）。

In [ ]:
#  Output:
# ∟LLMが質問と検索されたデータを含むプロンプトを使用して答えを生成する。

In [ ]:
#  Answer:
# ∟LLMが生成した回答が得られる。

In [ ]:
## Q&A
### ライブラリの読み込み
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import faiss

### FAISS vectorのロード
# faiss_indexをpickleファイルに保存
# vectoreStore = faiss_index.load_local("../database/faiss_index/", embeddings)
## Retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":5})

# promptの作成
template = """contextに従って回答してください:
{context}

質問: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# modelの作成
model = ChatOpenAI(model="gpt-3.5-turbo")

# chainを作成

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("On which datasets does GPT-3 struggle?")

'GPT-3 struggles on datasets related to common sense physics, as well as on tasks like SuperGLUE BoolQ, CB, COPA, RTE, WiC, WSC, MultiRC, and ReCoRD. It also faces difficulties on tasks that involve comparing the use of a word in two sentences (WIC), determining if one sentence implies another (ANLI), and certain reading comprehension tasks (e.g. QuAC and RACE).'